In [ ]:
import psycopg2

import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from IPython.display import display

## Establish a connection with the database

In [ ]:
params = {
  'database': 'postgres',
  'user': 'postgres',
#   'password': '',                                        ####### removed for security purposes
  'host': 'db.ozfxxdvqaykhwqfijquv.supabase.co',
  'port': 5432
}

conn = psycopg2.connect(**params)
cursor = conn.cursor()

In [ ]:
# print all the tables
cursor.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")

for table in cursor.fetchall():
    print(table)

In [ ]:
# dump all the tables to a local folder
# This takes a minute depending on internet quality. Consider just asking Judge 1 for the zipped folder instead.
cursor.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")

for table in np.array(cursor.fetchall())[::-1]:
    print(table)
    if table[0] != 'prototypes':
        df = pd.read_sql_query(f'select * from "{table[0]}"',con=conn)
        df.to_csv(f'csv_dump/{table[0]}.csv')

## Load survey questions

In [ ]:
entries = pd.read_csv('csv_dump/study_entries.csv')
entries = entries[entries['updated_at'].notna()].copy() # get rid of entries where nothing happened
survey = pd.read_csv('csv_dump/study_survey_questions.csv')

In [ ]:
# merge prolific_id into survey
survey['prolific_id'] = ""
for study_id in survey['study_id']:
    survey.loc[survey['study_id']==study_id, 'prolific_id'] = entries[entries['id']==study_id]['prolific_pid'].item()

## Load the study results (graded performance from users)

In [ ]:
df1 = pd.read_csv('csv_dump/user_study_scores-Grader1.csv')
df1['grader'] = 'Grader1'

df2 = pd.read_csv('csv_dump/user_study_scores-Grader2.csv')
df2['grader'] = 'Grader2'

# some sanity checks
assert all([id1 == id2 for (id1, id2) in zip(df1['study_id'].values, df2['study_id'].values)])
assert all([col1 == col2 for (col1, col2) in zip(df1.columns, df2.columns)])

df1 = df1.drop(columns=['p3t1:complexity', 'p3t1:new commands', 'p3t1:line count'])
df2 = df2.drop(columns=['p3t1:complexity', 'p3t1:new commands', 'p3t1:line count'])

In [ ]:
df = pd.concat([df1, df2])

In [ ]:
# convert step count to 0 or 1
df['p1t3:steps'] = df['p1t3:steps']==3
df['p1t3:steps'] = df['p1t3:steps'].astype(int)

In [ ]:
# melt the dataframe (convert to longform)

# helper mappings
def section_name_map(x):
    mapping = {
        'p1' : 'Writing',
        'p2' : 'Reading'
    }
    return mapping[x]

def task_name_map(x):
    mapping = {
        'p1t1' : 'Writing task 1',
        'p1t2' : 'Writing task 2',
        'p1t3' : 'Writing task 3',
        'p2t1' : 'Reading task 1',
        'p2t2' : 'Reading task 2',
        'p2t3' : 'Reading task 3',
    }
    return mapping[x]

id_vars = ['study_id', 'grader']
value_vars = set(df1.columns).difference(set(id_vars))
dfl = pd.melt(df, id_vars=id_vars, value_vars=value_vars)
dfl['task_id'] = dfl['variable'].apply(lambda x : x.split(':')[0])
dfl['property'] = dfl['variable'].apply(lambda x : x.split(':')[-1])
dfl['section'] = dfl['task_id'].apply(lambda x : x[:2])
dfl['task name'] = dfl['task_id'].apply(task_name_map)
dfl['section name'] = dfl['section'].apply(section_name_map)
dfl = dfl[dfl['section']!='p3'].copy()

dfl.head()

In [ ]:
# peak at differences between Grader2 and Grader1 scores
diff_df = dfl[dfl['grader']=='Grader1'].copy()
p_df = dfl[dfl['grader']=='Grader2'].copy()
diff_df['different'] = ~(diff_df['value'].values == p_df['value'].values)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12,6))
sns.barplot(x='variable', y='different', ci=None, ax=ax, data=diff_df)
plt.xticks(rotation=90)

disagreements = diff_df['different'].sum()
total_agreement = 1 - (disagreements / len(diff_df))
print(f'disagreements between Grader1 and Grader2:{disagreements}.')
print(f'disagreements between Grader1 and Grader2:{total_agreement}.')
None

In [ ]:
# let's just average Grader2 and Grader1 scores for now
dfl = dfl.groupby(['study_id', 'variable', 'task name', 'property', 'section name']).mean().reset_index()

In [ ]:
# merge with some survey questions..
dfl['education'] = -1
dfl['familiarity'] = -1
dfl['experience'] = -1
dfl['duration'] = -1
dfl['db_id'] = ""

for study_id in dfl['study_id'].unique():
    dfl.loc[dfl['study_id']==study_id, 'education'] = survey[survey['prolific_id'] == study_id]['education'].item()
    dfl.loc[dfl['study_id']==study_id, 'familiarity'] = survey[survey['prolific_id'] == study_id]['familiarity'].item()
    dfl.loc[dfl['study_id']==study_id, 'experience'] = survey[survey['prolific_id'] == study_id]['experience'].item()
    dfl.loc[dfl['study_id']==study_id, 'duration'] = entries[entries['prolific_pid'] == study_id]['duration'].max()/60
    dfl.loc[dfl['study_id']==study_id, 'db_id'] = entries[entries['prolific_pid'] == study_id]['id'].item()
    
assert -1 not in dfl['education'].values
assert -1 not in dfl['familiarity'].values
assert -1 not in dfl['experience'].values
assert -1 not in dfl['duration'].values
assert "" not in dfl['db_id'].values

In [ ]:
dfl.to_csv('csv_dump/processed_study_results.csv')